# **ETAPA 2 - IMPORTAÇÃO DAS BASES DE DADOS**

<br>

---

<br>

Agora será feito a importação dos dados para a camada especificada do Data LakeHouse que já está pronta.

*`Esse é um modelo modular, complete as informações necessárias nos trechos que estão destacados em vermelho assim como esse, seguindo o padrão snake_case.`*

<br> 

***AVISO**: Esse Notebook foi feito com base na estrutura do Databricks Free Edition, que utiliza catálogos.*

<br><br>

---

<br>

### Parte 1 - **Importação das Bibliotecas Necessárias**

In [0]:
# import pyspark 
import urllib
import datetime

<br>

---

<br>

### Parte 2 - **Importação dos Arquivos para o Data Lakehouse**

`Insira nas variáveis:` <br>
`--> nome_datalakehouse --> nome do Data Lakehouse destino` <br>

`--> nome_camada --> o nome da camada destino dos dados totalmente brutos` <br>
`--> nome_volume_atual --> nome do volume onde ficaram os dados atualizados` <br>
`--> nome_volume_historico --> nome do volume onde ficaram o histórico dos dados` <br>

In [0]:
nome_datalakehouse = "dataexperts"

nome_camada = "landing_zone"
nome_volume_atual = "vendas_atual"
nome_volume_historico = "vendas_historico"

O código a seguir cria os volumes se eles ainda não existirem:

In [0]:
spark.sql(f"""
CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada}.{nome_volume_atual}
""")

spark.sql(f"""
CREATE VOLUME IF NOT EXISTS {nome_datalakehouse}.{nome_camada}.{nome_volume_historico}
""")

`Insira nos vetores todos os nomes dos arquivos que deseja importar e a origem deles, seguindo a mesma ordem:`

In [0]:
arquivos = [
    "categoria_produto.csv",
    "cliente.csv",
    "data.csv",
    "localidade.csv",
    "produto.csv",
    "vendas_part1.csv",
    "vendas_part2.csv",
    "vendas_part3.csv",
    "vendas_part4.csv"
]

origem_arquivos = [
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
    "https://raw.githubusercontent.com/andrerosa1977/dataexperts2026/main/",
]

O código a seguir verifica se o arquivo já existe no Data Lakehouse:
- Se sim, apenas pula para o próximo download (se ainda exitir arquivos na fila em espera)
- Se não, tenta baixar o arquivo da origem.
Logo após, caso o download dê certo ou não, passa para a próximo download.

In [0]:
destino_atual = f"/Volumes/{nome_datalakehouse}/{nome_camada}/{nome_volume_atual}/"
destino_historico = f"/Volumes/{nome_datalakehouse}/{nome_camada}/{nome_volume_historico}/"

for arquivo, origem in zip(arquivos, origem_arquivos):
    entidade = arquivo.replace(".csv", "")
    
    try:
        arquivos_no_volume = dbutils.fs.ls(destino_atual)
        
        for arq in arquivos_no_volume:
            if arq.name.startswith(f"{entidade}_") and arq.name.endswith(".csv"):
                
                nome_sem_extensao = arq.name.replace(".csv", "")
                partes = nome_sem_extensao.split("_")
                data_str = partes[-2] 
                
                ano_arq = data_str[0:4]
                mes_arq = data_str[4:6]
                dia_arq = data_str[6:8]
                
                pasta_historico_real = f"{destino_historico}{entidade}/{ano_arq}/{mes_arq}/{dia_arq}/"
                
                dbutils.fs.mkdirs(pasta_historico_real)
                
                caminho_historico = f"{pasta_historico_real}{arq.name}"
                dbutils.fs.mv(arq.path, caminho_historico)
                
                print(f"[INFO] Histórico criado e arquivo movido: {caminho_historico}")
                
    except Exception as e:
        pass

    horario_brasil = datetime.timezone(datetime.timedelta(hours=-3))
    now = datetime.datetime.now(datetime.timezone.utc).astimezone(horario_brasil)
    timestamp_novo = now.strftime("%Y%m%d_%H%M%S")
    
    nome_arquivo_novo = f"{entidade}_{timestamp_novo}.csv"
    caminho_novo = f"{destino_atual}{nome_arquivo_novo}"

    try:
        urllib.request.urlretrieve(origem + arquivo, caminho_novo)
        print(f"[INFO] Novo arquivo {nome_arquivo_novo} baixado com sucesso.")
    except Exception as e:
        print(f"[ERRO] Falha ao baixar {arquivo}: {e}")

<br>

---

<br>

### **Resultado Final**

O código a seguir mostra todos os arquivos mais recentes salvos:

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

display(dbutils.fs.ls(destino_atual))


O código a seguir mostra o histórico de todas as versões dos arquivos salvos:

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

display(dbutils.fs.ls(destino_historico))

O código a seguir mostra um arquivo como exemplo:

In [0]:
# Código está comentado por ser apenas uma demonstração. Para visualizar o resultado, retire o identificador de comentário e execute novamente

nome_arquivo = arquivos[0].split('.')[0]
arquivos_no_diretorio = dbutils.fs.ls(destino_atual)

arquivo_alvo = None

for arquivo in arquivos_no_diretorio:
    if arquivo.name.startswith(nome_arquivo):
        arquivo_alvo = arquivo.path
        break

if arquivo_alvo:
    dataframe_arquivo_exemplo = spark.read.csv(arquivo_alvo, header=True, inferSchema=True)
    display(dataframe_arquivo_exemplo)
else:
    print("[INFO] Arquivo não encontrado!")